In [80]:
from utils import *

In [2]:
x = []
y = []
height = []
width = []

image = []

for i in range (1,61):
    tree = ET.parse('hand_detect/openxml/hframe'+str(i)+'.xml')
    root = tree.findall("object/bndbox/")
    im = cv2.imread(tree.findall("path")[0].text)

    top = int(root[1].text)
    bottom = int(root[3].text)
    left = int(root[0].text)
    right = int(root[2].text)

    image.append(im)
    x.append(int((left+right)/2))
    y.append(int((top+bottom)/2))
    height.append(int(bottom-top))
    width.append(int(right-left))

Z = np.ones(60)
X = np.array(x)
Y = np.array(y)
H = np.array(height)
W = np.array(width)
Image = np.array(image)/255


In [3]:
Image.shape

(60, 386, 600, 3)

In [4]:
box = np.vstack([Z,X,Y,H,W]).T

In [5]:
box.shape

(60, 5)

In [6]:
Nimg = []
for i in range (1,41):
    img = cv2.imread("hand_detect/frame"+str(i)+".jpg")
    Nimg.append(img)
Nhand = np.array(Nimg)/255
Z_0 = np.zeros(40).reshape(40,1)

In [7]:
Nhand.shape

(40, 386, 600, 3)

In [8]:
no_Hand = np.random.random_integers(0,600,size=(40,4))
nohand = np.hstack([Z_0,no_Hand])

C:\Users\AADITY~1\AppData\Local\Temp/ipykernel_972/1709062125.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 600 + 1) instead
  no_Hand = np.random.random_integers(0,600,size=(40,4))


In [9]:
nohand.shape

(40, 5)

In [10]:
data_y = np.vstack([box,nohand])

In [11]:
data_x = np.vstack([Image,Nhand])
data_x.shape

(100, 386, 600, 3)

In [12]:
Datax = data_x.reshape(100,-1)

In [13]:
data = np.hstack([Datax,data_y])

In [14]:
data.shape

(100, 694805)

In [52]:
headers = []
for i in range (Datax.shape[1]):
    headers.append("pixel"+str(i))
headers.append("object")
headers.append("xpos")
headers.append("ypos")
headers.append("height")
headers.append("width")

In [56]:
headers[-6]

'pixel694799'

In [57]:
Data = pd.DataFrame(data,columns=headers)

In [58]:
print(Data.columns[0:2])

Index(['pixel0', 'pixel1'], dtype='object')


In [59]:
DataNew = Data.sample(frac=1).reset_index(drop=True)
DataNew = DataNew.sample(frac=1).reset_index(drop=True)
DataNew = DataNew.sample(frac=1).reset_index(drop=True)
DataNew = DataNew.sample(frac=1).reset_index(drop=True)

In [60]:
DataNew.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel694795,pixel694796,pixel694797,pixel694798,pixel694799,object,xpos,ypos,height,width
0,0.603922,0.600000,0.607843,0.603922,0.600000,0.607843,0.603922,0.600000,0.607843,0.603922,...,0.411765,0.411765,0.419608,0.411765,0.411765,0.0,460.0,36.0,319.0,390.0
1,0.843137,0.843137,0.843137,0.843137,0.843137,0.843137,0.854902,0.847059,0.847059,0.854902,...,0.619608,0.615686,0.627451,0.619608,0.615686,0.0,274.0,64.0,380.0,305.0
2,0.658824,0.639216,0.647059,0.658824,0.639216,0.647059,0.658824,0.639216,0.647059,0.658824,...,0.086275,0.078431,0.133333,0.086275,0.078431,1.0,292.0,123.0,223.0,193.0
3,0.611765,0.592157,0.596078,0.611765,0.592157,0.596078,0.611765,0.592157,0.596078,0.611765,...,0.082353,0.074510,0.129412,0.082353,0.074510,1.0,455.0,264.0,190.0,165.0
4,0.858824,0.843137,0.839216,0.858824,0.843137,0.839216,0.858824,0.843137,0.839216,0.858824,...,0.611765,0.607843,0.631373,0.611765,0.607843,0.0,321.0,423.0,258.0,556.0


In [65]:
datan = np.array(DataNew)

In [72]:
Xdata = datan[:,:-5].reshape(-1,386,600,3)
Ydata = datan[:,-5:].reshape(-1,5)

In [93]:
def residual(input):
    inp = Convolution2D(kernel_size=(1,1),filters=128,activation=relu,padding="same")(input)
    x = Convolution2D(kernel_size=(3,3),filters=32,padding="same",kernel_initializer="normal",activation=relu)(input)
    x = Convolution2D(kernel_size=(3,3),filters=64,padding="same",kernel_initializer="normal",activation=relu)(x)
    x = Convolution2D(kernel_size=(3,3),filters=128,padding="same",kernel_initializer="normal",activation=relu)(x)
    x = Add()([x,inp])
    output = MaxPool2D(pool_size=(2,2),strides=(2,2))(x)

    return output


In [94]:
print(Xdata.shape,Ydata.shape)

(100, 386, 600, 3) (100, 5)


In [95]:
inp = Input(shape=(386,600,3))
for i in range (4):
    intermidiate = residual(inp1)
    inp1 = intermidiate

ou = Flatten()(intermidiate)
out = Dense(units=64,activation=relu,kernel_initializer="normal")(ou)
output = Dense(units=5,activation=softmax)